# Set Up

In [ ]:
%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import bepy as be
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, NMF, DictionaryLearning
import matplotlib.pyplot as plt
import seaborn as sns

sset = be.SampleSet()

# Load Data

In [ ]:
gridSize = 30
sampName = 'PT27'
path = 'D:\\Data\\Lee\\pt27_007\\01-15-2019\\'

samp = be.Sample(path, gridSize)

sset.addsample(samp, sampName, gridSize)

# Basic Plots

## BE Scan Plots

In [ ]:
lims = np.array([[0,100,0.325,0],[800,270,0.375,300]])
samp._gridmeasurements['Vert'].plot(lims=lims, plotgroup=1)

In [ ]:
lims = np.array([[0,0,0.745,0],[30,300,0.800,1200]])
samp._linemeasurements['Lat'].plot(lims=lims, plotgroup=1)

## SSPFM Plot

In [ ]:
samp._gridmeasurements['SSPFM'].plot(pointNum=100)

## Relaxation Plot

In [ ]:
samp._gridmeasurements['Relax'].plot(pointNum=None, InOut=1,insert=np.inf)

In [ ]:
mask = test.columns.get_level_values(1) == 0.0
olphase = test.T[mask].T
xax = olphase.columns.get_level_values(3)

In [ ]:
mph = olphase.mean()

In [ ]:
phase = olphase.loc[75,:].values

In [ ]:
offset = np.pi/2 - (np.mean(mph)+np.std(mph))

In [ ]:
fig = plt.figure()
plt.plot(xax,mph+offset)
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(np.diff(phase))
plt.show()

In [ ]:
newPh = np.full(phase.shape,0)

add = np.where(np.diff(phase) < -1.8*np.pi)
sub = np.where(np.diff(phase) > 1.8*np.pi)
good = np.where(np.abs(np.diff(phase))<=1.8*np.pi)

newPh[good] = phase[good]
newPh[add] = phase[add] + 2*np.pi
newPh[sub] = phase[sub] - 2*np.pi

#for i in np.where(np.abs(np.diff(phase))>1.8*np.pi)[0]:
#    val = np.diff(phase)[i]
#    print(val)
#    if val > 0:
#        newPh[i] = phase[i] - 2*np.pi
#    else:
#        newPh[i] = phase[i] + 2*np.pi
#    print(newPh[i])

In [ ]:
add

In [ ]:
fig = plt.figure()
plt.plot(xax,newPh)
plt.show()

In [ ]:
samp._gridmeasurements['Relax'].plot(InOut=1.0,insert=np.inf, pointNum=None)

In [ ]:
nmfmodel=NMF(n_components=3, tol=1e-6, max_iter=500)
maps,comps=sset.fit(nmfmodel, sampstack=[sampName], measstack=['Relax'], varstack=['Amp'])

In [ ]:
sset._analysis.plot()

In [ ]:
data = samp._linemeasurements['Scan']._data['Phase'].applymap(lambda x: x-2*np.pi if x>np.pi else x-0)
plt.imshow(data,vmin=0,vmax=np.pi,cmap='nipy_spectral')
plt.plot
plt.colorbar()

In [ ]:
samp._linemeasurements['Scan']._data['Phase'].mean().mean()

# Temp FORC

In [ ]:
data = pd.read_csv(path+'FORC\\'+'shofit.csv')

In [ ]:
inout = data['InOut'] == 0

for i in range(int(data['PlotGroup'].max())):

    mask = np.logical_and(inout, data['PlotGroup'] == i)

    amp = data['Amp'].values[np.where(mask)]
    ph = data['Phase'].values[np.where(mask)]
    loop = amp*np.cos(ph)
    dc = data['DC'].values[np.where(mask)]
    
    plt.plot(10*dc, -loop)

plt.show()

In [ ]:
inout = data['InOut'] == 0
mask = np.logical_and(inout, data['PlotGroup'] == int(data['PlotGroup'].max()))

amp = data['Amp'].values[np.where(mask)]
ph = data['Phase'].values[np.where(mask)]

return_loops = np.full((int(data['PlotGroup'].max()), int(len(amp)/2)),np.nan)

plt.figure()

for i in range(int(data['PlotGroup'].max())):
    
    mask = np.logical_and(inout, data['PlotGroup'] == i)

    amp = data['Amp'].values[np.where(mask)]
    ph = data['Phase'].values[np.where(mask)]
    loop = amp*np.cos(ph)
    dc = data['DC'].values[np.where(mask)]
    
    loop = loop[int(len(loop)/2):]
    dc = dc[int(len(dc)/2):]
    
    return_loops[i, (np.arange(0,len(loop)))] = -np.flip(loop,0)
    
    plt.plot(10*dc, -loop)

plt.show()

return_loops = np.flip(return_loops,1)

plt.figure()

for i in range(int(data['PlotGroup'].max())):
    plt.plot(return_loops[i,:])
    plt.show()

In [ ]:

derivative_horiz = np.full((np.shape(return_loops)[0]-1,np.shape(return_loops)[1]-1), np.nan)

for i in range(np.shape(return_loops)[0]-1):
    for j in range(np.shape(return_loops)[1]-1):
        derivative_horiz[i,j] = (return_loops[i,j-1]+return_loops[i,j+1])/0.378

priesach = np.full((np.shape(derivative_horiz)[0]-1,np.shape(derivative_horiz)[1]-1), np.nan)

for i in range(np.shape(derivative_horiz)[0]-1):
    for j in range(np.shape(derivative_horiz)[1]-1):
        priesach[i,j] = (derivative_horiz[i,j-1]+derivative_horiz[i,j+1])/0.378
    
plt.imshow(priesach,cmap='nipy_spectral')
plt.colorbar()
plt.show()